# Исследование надежности заемщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Обзор данных

Составим первое представление о данных. Для этого импортируем библиотеку `pandas`, прочитаем файл data.csv и выведем первые 10 строк:

In [1]:
import pandas as pd                      # импорт библиотеки pandas

df = pd.read_csv('data.csv')   # чтение файла с данными и сохранение в df
df.head(10)                              # вывод первых 10 строк датасета

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Получим общую информацию о таблице:

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


В таблице 12 столбцов. Согласно документации к данным:

* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

Количество значений в столбцах различается. Значит в данных есть пропущенные значения. 

Также видны аномалии в данных:
* в столбцах `children` и `days_employed` нехарактерные для них отрицательные значения, присутствуют выбросы (максимум сильно отличается от 3 квартиля);
* в столбце `dob_years`, отражающем возраст клиента, значение минимума равно 0.



## Шаг 2.1 Проверка данных на аномалии и исправления.

### `income_type`

Выведем уникальные значения и их количество в столбце `income_type` (тип занятости):

In [4]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Аномалий не выявлено.

### `days_employed`

При выводе первых строк датасета становятся очевидными аномалии в столбце `days_employed`: трудовой стаж выражен отрицательным количеством дней или приближается к тысячи лет.

Можно предположить, что появление аномальных значений трудового стажа связано с типом дохода (`income_type`).

Выведем типы дохода, для которых стаж отрицателен:

In [5]:
df[df['days_employed'] < 0]['income_type'].value_counts()

сотрудник          10014
компаньон           4577
госслужащий         1312
студент                1
предприниматель        1
в декрете              1
Name: income_type, dtype: int64

Равен 0:

In [6]:
df[df['days_employed'] == 0]['income_type'].value_counts()

Series([], Name: income_type, dtype: int64)

Положителен:

In [7]:
df[df['days_employed'] > 0]['income_type'].value_counts()

пенсионер      3443
безработный       2
Name: income_type, dtype: int64

Причем минимумом среди положительных значений будет 328728 дня $\approx$ 900 лет:

In [8]:
df[df['days_employed'] > 0]['days_employed'].min()

328728.72060451825

Действительно, для пенсионеров и безработных стаж положителен и выражается в очень больших числах, для людей с другими типами дохода — отрицателен. Можно предположить, что для разных типов дохода данные были выгружены из разных источников, а соответственно и в разном формате (знак тире превратился в минус, а в положительных значениях неправильно проставлена запятая). Это и стало причиной аномалий.

Каждое положительное значение стажа разделим на 100:

In [9]:
df.loc[df['days_employed'] > 0, 'days_employed'] = df.loc[df['days_employed'] > 0, 'days_employed'] / 100

Отрицательные значения заменим положительными:

In [10]:
df['days_employed'] = df['days_employed'].abs()

Проверим результат. Выведем первые 10 строк `days_employed`:

In [11]:
df['days_employed'].head(10)

0    8437.673028
1    4024.803754
2    5623.422610
3    4124.747207
4    3402.660720
5     926.185831
6    2879.202052
7     152.779569
8    6929.865299
9    2188.756445
Name: days_employed, dtype: float64

Аномальные значения исправлены.

### `children`

Выведем уникальные значения и их количество в столбце `children` (количество детей в семье):

In [12]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Значение -1, вероятно, появилось в результате опечатки при вводе данных. Заменим его на 1:

In [13]:
df['children'] = df['children'].replace(-1, 1)

Однозначно определить, какое значение должно быть на месте 20 невозможно, но количество его упоминаний меньше 1% от выборки (0.35%), поэтому удаление строк не сильно отразится на результатах исследования.

In [14]:
df = df.loc[df['children'] != 20].reset_index(drop=True)

Проверим результат:

In [15]:
df['children'].value_counts()

0    14149
1     4865
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

Аномальные значения исправлены.

### `dob_years`

Проверим минимальное и максимальное значения для возраста клиента:

In [16]:
print(df['dob_years'].min())
print(df['dob_years'].max())

0
75


Возраст клиента не может быть меньше 18 и тем более равен 0. Проверим, сколько всего таких значений в таблице:

In [17]:
df[df['dob_years'] < 18]['dob_years'].count()

100

Значения, которые меньше 18, в сумме с ранее удаленными строками составляют менее 1% от выборки, поэтому удаление строк не сильно отразится на результатах ислледования.

In [18]:
df = df.loc[df['dob_years'] >= 18].reset_index(drop=True)

Проверим результат:

In [19]:
df['dob_years'].min()

19

### `gender`

Проверим уникальные значения и их количество в столбце `gender`:

In [20]:
df['gender'].value_counts()

F      14118
M       7230
XNA        1
Name: gender, dtype: int64

Удалим строку со значением `XNA`:

In [21]:
df = df[df['gender'] != 'XNA']

Проверим результат:

In [22]:
df['gender'].value_counts()

F    14118
M     7230
Name: gender, dtype: int64

### `total_income`

Проверим, что в столбце нет отрицательных значений:

In [23]:
df[df['total_income'] < 0]['total_income'].count()

0

Аномалий не выявлено.

## Шаг 2.2 Заполнение пропусков

Посчитаем, сколько в таблице пропущенных значений:

In [24]:
df.isna().sum()

children               0
days_employed       2155
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2155
purpose                0
dtype: int64

Посчитаем, сколько из них пропущены одновременно в `days_employed` и `total_income`.

In [25]:
df[df['days_employed'].isna() & df['total_income'].isna()].shape[0]

2155

Число пропущенных значений в `days_employed` и `total_income` равно, причем они появляются в двух столбцах одновременно. Это значит, что пропуски нельзя отнести к абсолютно случайным и причина их появления совпадает.

Проверим, какую долю составляют пропущенные значения от всех:

In [26]:
f"{(df.isna().sum()['days_employed'] / df['days_employed'].count()):.0%}"

'11%'

Такое количество пропусков может повлиять на результат исследования. Заменим пропущенные значения в зависимости от типа данных.

Столбцы `days_employed` и `total_income` содержат информацию о трудовом стаже и ежемесячном доходе. Тип данных — количественный, причем в данных могут быть выдающиеся значения. Следовательно, для заполнения пропусков используем характерные значения, а именно — медиану. Для повышения точности заполнения пропущенных данных учтем тот факт, что медианный ежемесячный доход будет различаться для разных типов занятости.

Таким образом, задача принимает следующий вид:

1. Каждое пропущенное значение в столбце `days_employed` заменить на медианное по столбцу.
1. Каждое пропущенное значение в столбце `total_income` заменить на медианное для типа занятости (`income_type`) в строке с этим значением.

In [27]:
# медианный трудовой стаж по столбцу
days_employed_median = df['days_employed'].median()

# медианный доход для каждого типа занятости
grouped_by_income_median = df.groupby('income_type')['total_income'].median()

# функция, которая принимает на вход строку датасета и меняет:
# трудовой стаж на медианный по столбцу
# ежемесячный доход на медианный для этого типа занятости
# и возвращает преобразованную строку
def fillna_by_median(row):
    row['days_employed'] = days_employed_median
    row['total_income'] = grouped_by_income_median[row['income_type']]    
    return row

# вызов функции для строк датасета с пропусками в days_employed
df[df['days_employed'].isna()] = df[df['days_employed'].isna()].apply(fillna_by_median, axis=1)

Убедимся, что не осталось пропусков:

In [28]:
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

## Шаг 2.3. Изменение типов данных.

Заменим вещественный тип данных в столбце `total_income` на целочисленный:

In [29]:
df['total_income'] = df['total_income'].astype('int')

Проверим результат:

In [30]:
df['total_income'].dtype

dtype('int64')

## Шаг 2.4. Удаление дубликатов.

Посчитаем явные дубликаты в таблице:

In [31]:
df.duplicated().sum()

54

Удалим явные дубликаты с удалением старых индексов и формированием новых:

In [32]:
df = df.drop_duplicates().reset_index(drop=True)

Проверим результат:

In [33]:
df.duplicated().sum()

0

Удалим неявные дубликаты в столбце `education`. 

Для начала выведем список уникальных значений:

In [34]:
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Чтобы избавиться от дубликатов, приведем все значения в столбце к нижнему регистру:

In [35]:
df['education'] = df['education'].str.lower()

Проверим результат:

In [36]:
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Проверим, есть ли неявные дубликаты в столбце `family_status`:

In [37]:
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Дубликатов нет.

## Шаг 2.5. Формирование дополнительных датафреймов-словарей, декомпозиция исходного датафрейма.

Чтобы было удобнее обращаться к данным об образовании и семейном положении клиента (по id), создадим словари по столбцам `education` и `family_status`.

Создание словаря с видами образования:

In [38]:
education_dict = df[['education_id', 'education']]                       # создание словаря с соответствием 'education_id' и 'education'
education_dict = education_dict.drop_duplicates().reset_index(drop=True) # удаление дубликатов и формирование новых индексов
education_dict                                                           # вывод словаря

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


Создание словаря по семейному положению:

In [39]:
family_status_dict = df[['family_status_id', 'family_status']]                   # создание словаря с соответствием 'family_status_id' и 'family_status'
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True) # удаление дубликатов и формирование новых индексов
family_status_dict                                                               # вывод словаря

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


Удалим столбцы `education` и `family_status` из исходного датафрейма, оставив только столбцы с id:

In [40]:
df = df.drop(columns=['education', 'family_status']) # удаление столбцов
df.head()                                            # вывод первых 5 строк датафрейма

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,3402.660720,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


## Шаг 2.6. Категоризация дохода.

Объединим данные о доходе клиентов в категории со следующими диапазонами:

* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'.

Значение категории сохраним в отдельном столбце `total_income_category`.

In [41]:
# функция, которая возвращает значение категории 
# в зависимости от диапазона, в который входит доход

def categorize_income(income):
    if income <= 30000:
        return 'E'
    if income <= 50000:
        return 'D'
    if income <= 200000:
        return 'C'
    if income <= 1000000:
        return 'B'
    return 'A'

# формирование столбца total_income_category вызовом функции к столбцу total_income
df['total_income_category'] = df['total_income'].apply(categorize_income)
# вывод первых 5 строк датасета
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,3402.660720,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


## Шаг 2.7. Категоризация целей кредита.

Определим категории, в которые могут объединены цели кредита. Для этого найдем все леммы в уникальных значениях столбца `purpose` и выделим те из них, которые по смыслу выражают цель кредита.

In [42]:
# импорт модуля из библиотеки для лемматизации
from pymystem3 import Mystem                 
m = Mystem()

#обявление списка для хранения лемм
all_lemmas = []

# лемматизация уникальных значений столбца purpose
# и добавление каждой леммы к общему списку,
# если ее еще там нет
for phrase in df['purpose'].unique():
    lemmatized_phrase = m.lemmatize(phrase)
    for lemma in lemmatized_phrase:
        if lemma not in all_lemmas:
            all_lemmas.append(lemma)

# вывод списка лемм
all_lemmas

['покупка',
 ' ',
 'жилье',
 '\n',
 'приобретение',
 'автомобиль',
 'дополнительный',
 'образование',
 'сыграть',
 'свадьба',
 'операция',
 'с',
 'на',
 'проведение',
 'для',
 'семья',
 'недвижимость',
 'коммерческий',
 'жилой',
 'строительство',
 'собственный',
 'подержать',
 'свой',
 'со',
 'заниматься',
 'сделка',
 'подержанный',
 'получение',
 'высокий',
 'профильный',
 'сдача',
 'ремонт']

Среди полученных лемм можно выделить: 'автомобиль', 'жилье', 'недвижимость', 'свадьба', 'образование'.

Что образует следующие категории целей: 

* 'операции с автомобилем';
* 'операции с недвижимостью';
* 'проведение свадьбы';
* 'получение образования'.

Напишем функцию, которая для каждого значения в столбце `purpose` выделит леммы и по ним определит, к какой категории отнести столбец.

In [43]:
# функция, которая принимает на вход строку,
# лемматизирует ее, проверяет наличие ключевых слов среди лемм
# и возвращает соответствующую категорию цели
def replace_lemmas(phrase):
    lemmatized_phrase = m.lemmatize(phrase)
    if 'автомобиль' in lemmatized_phrase:
        return 'операции с автомобилем'    
    if 'жилье' in lemmatized_phrase or 'недвижимость' in lemmatized_phrase:
        return 'операции с недвижимостью'
    if 'свадьба' in lemmatized_phrase:
        return 'проведение свадьбы'
    if 'образование' in lemmatized_phrase:
        return 'получение образования'
    return phrase

# формирование столбца purpose_category вызовом функции к столбцу purpose
df['purpose_category'] = df['purpose'].apply(replace_lemmas)

Проверим, для всех ли значений определена категория (или какие-то значения остались неизменными, потому что в них не были найдены заданные леммы):

In [44]:
df['purpose_category'].unique()

array(['операции с недвижимостью', 'операции с автомобилем',
       'получение образования', 'проведение свадьбы'], dtype=object)

Все значения категоризованы.

## Ответы на вопросы.

### Вопрос 1:

Есть ли зависимость между количеством детей и возвратом кредита в срок?

Построим сводную таблицу с группировкой по количеству детей и столбцами:

* `0` - число клиентов, не имевших задолженности по кредиту.
* `1` - число клиентов, имевших задолженности.
* `sum` - общее число клиентов.
* `ratio` - доля клиентов, имевших задолженности, среди всех клиентов.

In [45]:
# функция, которая принимает на вход строку и
# по значениям в столбцах '1' и '0'
# высчитывает sum и ratio

def find_sum_ratio(row):
    row['sum'] = row[0] + row[1]
    row['ratio'] = row[1] / row['sum']
    return row

# построение сводной таблицы
children_pivot = df.pivot_table(index='children', columns='debt',values='purpose', aggfunc='count')
# вызов функции find_sum_ratio для датафрейма
children_pivot = children_pivot.apply(find_sum_ratio, axis=1)

children_pivot

debt,0,1,sum,ratio
children,,,,
0,12979.0,1058.0,14037.0,0.075372
1,4398.0,442.0,4840.0,0.091322
2,1845.0,194.0,2039.0,0.095145
3,301.0,27.0,328.0,0.082317
4,37.0,4.0,41.0,0.097561
5,9.0,NaN,NaN,NaN


Нельзя утверждать, что доля должников растет с увеличением числа детей, так как среди клиентов с 5 детьми вообще нет должников, с 3 — должников меньше чем с 1 или 2. На результат влияет количество клиентов с 4-5 детьми. Выборка слишком маленькая, чтобы делать выводы.

Однако можно выдвинуть гипотезу, что возврат кредита в срок зависит не от количества детей, а от их наличия. Построим аналогичную сводную таблицу с группировкой по наличию детей:

In [46]:
# формирование столбца, который принимает значение
# 1, если дети есть
# 0, если детей нет
df['with_children'] = df['children'].apply(lambda x: 1 if x > 0 else 0)

# построение сводной таблицы
with_children_pivot = df.pivot_table(index='with_children', columns='debt',values='purpose', aggfunc='count')
# вызов функции find_sum_ratio для датафрейма
with_children_pivot = with_children_pivot.apply(find_sum_ratio, axis=1)

with_children_pivot

debt,0,1,sum,ratio
with_children,,,,
0,12979.0,1058.0,14037.0,0.075372
1,6590.0,667.0,7257.0,0.091911


### Вывод 1:

Зависимость между количеством детей и возвратом кредита в срок не наблюдается. 

Однако доля должников среди клиентов с детьми больше, чем доля должников среди клиентов без детей.

### Вопрос 2:

Есть ли зависимость между семейным положением и возвратом кредита в срок?

Построим сводную таблицу с группировкой по семейному положению и столбцами, аналогичными предыдущему заданию:

In [47]:
# построение сводной таблицы
family_pivot = df.pivot_table(index='family_status_id', columns='debt',values='purpose', aggfunc='count')
family_pivot = family_pivot.merge(family_status_dict, on='family_status_id').set_index(['family_status_id', 'family_status'])

# вызов функции find_sum_ratio для датафрейма
family_pivot = family_pivot.apply(find_sum_ratio, axis=1)
family_pivot

,,0,1,sum,ratio
family_status_id,family_status,,,,
0,женат / замужем,11323.0,924.0,12247.0,0.075447
1,гражданский брак,3746.0,383.0,4129.0,0.092759
2,вдовец / вдова,888.0,62.0,950.0,0.065263
3,в разводе,1099.0,84.0,1183.0,0.071006
4,Не женат / не замужем,2513.0,272.0,2785.0,0.097666


### Вывод 2:

Меньшая доля должников приходится на тех, кто состоит или состоял в официально зарегистрированном браке (категории "женат/замужем", "вдовец/вдова", "в разводе"). Клиенты, не состоящие в браке или состоящие в гражданском браке, имеют задолженности чаще.

### Вопрос 3:

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Построим сводную таблицу с группировкой по ежемесячному доходу и столбцами, аналогичными предыдущему заданию:

In [48]:
# построение сводной таблицы
income_pivot = df.pivot_table(index='total_income_category', columns='debt',values='purpose', aggfunc='count')

# вызов функции find_sum_ratio для датафрейма
income_pivot = income_pivot.apply(find_sum_ratio, axis=1)

income_pivot

debt,0,1,sum,ratio
total_income_category,,,,
A,23.0,2.0,25.0,0.080000
B,4642.0,353.0,4995.0,0.070671
C,14557.0,1347.0,15904.0,0.084696
D,327.0,21.0,348.0,0.060345
E,20.0,2.0,22.0,0.090909


Отсортируем категории по доле должников:

In [52]:
income_pivot.sort_values(by='ratio', ascending=False)

debt,0,1,sum,ratio
total_income_category,,,,
E,20.0,2.0,22.0,0.090909
C,14557.0,1347.0,15904.0,0.084696
A,23.0,2.0,25.0,0.080000
B,4642.0,353.0,4995.0,0.070671
D,327.0,21.0,348.0,0.060345


### Вывод 3:

Доля должников среди клиентов с самым низким доходом действительно наибольшая (0.09). Но зависимость между уровнем дохода и возвратом кредита в срок не наблюдается. Так, люди с высшим доходом на третьем месте среди должников, а клиенты с доходом, близким к низкому (30001-50000) на последнем с долей должников 0.06.

Возможно, важные данные в столбце `total_income` были утеряны при заполнении пропусков медианным значением.

### Вопрос 4:

Как разные цели кредита влияют на его возврат в срок?

Построим сводную таблицу с группировкой по цели кредита и столбцами, аналогичными предыдущему заданию:

In [49]:
# построение сводной таблицы
purpose_pivot = df.pivot_table(index='purpose_category', columns='debt',values='purpose', aggfunc='count')

# вызов функции find_sum_ratio для датафрейма
purpose_pivot = purpose_pivot.apply(find_sum_ratio, axis=1)

purpose_pivot

debt,0,1,sum,ratio
purpose_category,,,,
операции с автомобилем,3873.0,398.0,4271.0,0.093187
операции с недвижимостью,9953.0,777.0,10730.0,0.072414
получение образования,3612.0,369.0,3981.0,0.092690
проведение свадьбы,2131.0,181.0,2312.0,0.078287


### Вывод 4:

Доля должников меньше среди тех, кто берет кредит для операций с недвижимостью или проведения свадьбы, и больше среди тех, кто берет его для операций с автомобилем или получения образования. Возможно, это связано с тем, что к покупке жилья люди относятся более ответственно, так как сумма заемных средств и срок кредита, как правило гораздо выше чем в остальных категориях, а свадебные кредиты возвращаются за счет подаренных денег.

## Общий вывод:

Способность потенциального заёмщика вернуть кредит банку зависит от:
* наличия детей;
* семейного положения;
* цели кредита.

Меньшая доля должников приходится на тех, у кого нет детей, и кто состоит/состоял в официально зарегистрированном браке. При этом кредит чаще возвращают в срок, если его берут для операций с недвижимостью или проведения свадьбы.

Чтобы точно определить, влияет ли уровень дохода на возврат кредита в срок, необходимо восстановить пропущенные значения и провести еще одно исследование.